In [1]:
from tqdm import tqdm
import pytdx
from pytdx.reader import TdxDailyBarReader, TdxFileNotFoundException,CustomerBlockReader,TdxLCMinBarReader
reader = TdxLCMinBarReader()
from pytdx.hq import TdxHq_API
import pandas as pd
import datetime
from czsc.objects import RawBar, Freq
from czsc.kline_generator import KlineGenerator
from czsc.io import read_pkl, save_pkl
import shutil

In [2]:
def kline_merge(file_f1,file_pkl):
    cid = file_f1[18:26]
    df = pd.read_csv(file_f1)
    bars = []
    for i, row in df.iterrows():
        bar = RawBar(symbol=cid, dt=pd.to_datetime(row['date']), id=i, freq=Freq.F1, open=row['open'],
                     close=row['close'], high=row['high'], low=row['low'], vol=row['volume'])
        bars.append(bar)
    kg = KlineGenerator(max_count=100000000)
    for bar in tqdm(bars[:50000], desc="merge kline"):
        kg.update(bar)
    # 保存 kg 到 pkl 文件
    save_pkl(kg, file_pkl)
    # 获取结果
    f1 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='1分钟', count=10000000)])
    f5 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='5分钟', count=10000000)])
    f15 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='15分钟', count=10000000)])
    f30 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='30分钟', count=10000000)])
    f60 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='60分钟', count=10000000)])
    d = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='日线', count=10000000)])
    f1.to_csv(f'E:\\data\\Tdx_kline\\{cid}_1f.csv')
    f5.to_csv(f'E:\\data\\Tdx_kline\\{cid}_5f.csv')
    f15.to_csv(f'E:\\data\\Tdx_kline\\{cid}_15f.csv')
    f30.to_csv(f'E:\\data\\Tdx_kline\\{cid}_30f.csv')
    f60.to_csv(f'E:\\data\\Tdx_kline\\{cid}_60f.csv')
    d.to_csv(f'E:\\data\\Tdx_kline\\{cid}_1d.csv')


def kline_merge_new(file_f1,file_pkl):
    cid = file_f1[18:26]
    df = pd.read_csv(file_f1)
    bars = []
    for i, row in df.iterrows():
        bar = RawBar(symbol=cid,dt=pd.to_datetime(row['date']), id=i, freq=Freq.F1, open=row['open'],
                     close=row['close'], high=row['high'], low=row['low'], vol=row['volume'])
        bars.append(bar)

    # 重新加载 kg
    kg = read_pkl(file_pkl)

    for bar in tqdm(bars, desc="merge kline new"):
        if bar.dt > kg.end_dt:
            kg.update(bar)

    # 获取结果
    f1 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='1分钟', count=10000000)])
    f5 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='5分钟', count=10000000)])
    f15 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='15分钟', count=10000000)])
    f30 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='30分钟', count=10000000)])
    f60 = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='60分钟', count=10000000)])
    d = pd.DataFrame([x.__dict__ for x in kg.get_kline(freq='日线', count=10000000)])
    f1.to_csv(f'E:\\data\\Tdx_kline\\{cid}_1f.csv')
    f5.to_csv(f'E:\\data\\Tdx_kline\\{cid}_5f.csv')
    f15.to_csv(f'E:\\data\\Tdx_kline\\{cid}_15f.csv')
    f30.to_csv(f'E:\\data\\Tdx_kline\\{cid}_30f.csv')
    f60.to_csv(f'E:\\data\\Tdx_kline\\{cid}_60f.csv')
    d.to_csv(f'E:\\data\\Tdx_kline\\{cid}_1d.csv')

In [3]:
if __name__ == '__main__':
    tdx_df = CustomerBlockReader().get_df('E:\data\TdxBak_20210630\\blocknew\\')
    RPS=tdx_df[tdx_df['blockname'] == '板块指数RPS'].code.tolist()
    for i in [RPS[0]]:
        RPS_file = f'E:\\new_jytyb 纶巾羽扇 0509\\vipdoc\\sh\\minline\\sh{i}.lc1'
        dstfile=  r'E:\data\test'
        shutil.copy(RPS_file,dstfile)
        reader.get_df(f'E:\\new_jytyb 纶巾羽扇 0509\\vipdoc\\sh\\minline\\sh{i}.lc1', 'rb').to_csv(f'E:\\data\\Tdx_kline\\sh{i}_1m.csv')
        file_f1 = f'E:\\data\\Tdx_kline\\sh{i}_1m.csv'
        file_pkl = f'E:\\data\\Tdx_kline\\sh{i}_1m.pkl'
    #     kline_merge(file_f1,file_pkl)
        kline_merge(file_f1,file_pkl)

merge kline: 100%|█████████████████████████████████████████████████████████████| 50000/50000 [00:13<00:00, 3607.35it/s]


In [8]:
[RPS[0]]

'880501'

merge kline: 100%|█████████████████████████████████████████████████████████████| 50000/50000 [00:17<00:00, 2806.57it/s]
